In [34]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

df = pd.read_csv('hsa_all_datapoints.txt', sep='\t')
#df.iloc[:,20:33]

In [35]:
df.head()

,cluster,avg_seq_len,MainBody_length,avg_GC_cont,avg_ATcont_after3p,sum_all_reads,num_gap_bars,scale_rate_5p,num_mismatches_8mer,num_mism_in_MB,...,hits_on_mature_miR,bracket-mirna-fraction,hairpin-size-estimate,loop-mirna-distance-estimate,loop-size-estimate,loops-in-hairpin,minimum-free-energy,mirna-bracket-discrepancy,mirna-bracket-majority,mirna-unmatched
0,0,23.133180,21,0.523485,0.662613,3041,0,0,0,0,...,YES,0.583333,56.0,1.0,12.0,1.0,-20.5,1.0,14.0,10.0
1,1,27.600000,28,0.434689,0.044447,15,0,0,0,0,...,NO,0.357143,52.0,0.0,4.0,1.0,-10.4,1.0,10.0,18.0
2,2,21.647289,21,0.359256,0.094773,2047,2,2,0,0,...,YES,0.952381,58.0,2.0,12.0,1.0,-30.3,1.0,20.0,1.0
3,3,25.458333,21,0.652219,0.250000,48,6,6,0,0,...,NO,0.518519,71.0,4.0,5.0,2.0,-20.6,1.0,14.0,13.0
4,4,22.420795,26,0.392090,0.064233,3472,1,1,0,2,...,YES,0.772727,72.0,14.0,4.0,1.0,-28.6,1.0,17.0,5.0


In [36]:
y = np.array(df.hits_on_mature_miR)
y[y == 'NO'] = 0
y[y == 'YES'] = 1
y = to_categorical(y)

In [37]:
del df['hits_on_mature_miR']

In [38]:
df.fillna(0)
print(len(df))

162791


In [39]:
X = np.array(df)
X

array([[  0.00000000e+00,   2.31331799e+01,   2.10000000e+01, ...,
          1.00000000e+00,   1.40000000e+01,   1.00000000e+01],
       [  1.00000000e+00,   2.76000000e+01,   2.80000000e+01, ...,
          1.00000000e+00,   1.00000000e+01,   1.80000000e+01],
       [  2.00000000e+00,   2.16472887e+01,   2.10000000e+01, ...,
          1.00000000e+00,   2.00000000e+01,   1.00000000e+00],
       ..., 
       [  1.05800000e+03,   1.60000000e+01,   1.60000000e+01, ...,
          8.57142857e-01,   6.00000000e+00,   9.00000000e+00],
       [  1.05900000e+03,   1.60000000e+01,   1.60000000e+01, ...,
          1.00000000e+00,   8.00000000e+00,   8.00000000e+00],
       [  1.06000000e+03,   1.60000000e+01,   1.60000000e+01, ...,
          5.55555556e-01,   5.00000000e+00,   7.00000000e+00]])

In [68]:
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.constraints import maxnorm

model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(X.shape[1],) ))
model.add(Dense(1000, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(500, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)

model.fit(X, y, callbacks=[early_stopping_monitor], validation_split=0.2, batch_size=300)

Train on 130232 samples, validate on 32559 samples
Epoch 1/10
130232/130232 [==============================] - 55s - loss: nan - acc: 0.7934 - val_loss: nan - val_acc: 0.8484
Epoch 2/10
130232/130232 [==============================] - 56s - loss: nan - acc: 0.7942 - val_loss: nan - val_acc: 0.8484
Epoch 3/10
130232/130232 [==============================] - 55s - loss: nan - acc: 0.7942 - val_loss: nan - val_acc: 0.8484
